<a href="https://colab.research.google.com/github/repitta/CienciaDeDadosEducacionais/blob/master/nota_das_provas_do_Lop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Expandir a tabela de provas para que cada questão das provas seja uma linha da tabela

[ ]
colunas   = ["idProva","idQuestao","titulo",'data']
provaNovo = pd.DataFrame(columns=colunas)
for i in range(len(tabProvas)):
  idProva   = tabProvas["_id"].iloc[i]
  titulo    = tabProvas["titulo"].iloc[i]
  data      = tabProvas["iniciou"].iloc[i]
  q         = tabProvas["questao"].iloc[i]
  for a in range(6):
    questao = q[a]
    dic = {"idProva": idProva ,"titulo":titulo, "idQuestao": questao ,"data":data}
    provaNovo = pd.concat([provaNovo, pd.DataFrame(dic, index=[0])],  ignore_index=True, sort=False )
​
Deletar as linhas que estão com NaN

[ ]
provaNovo = provaNovo.dropna(axis=0, how='any')
print(provaNovo.shape)
provaNovo.info()
Criar o campo quantidade de questoes por prova

[ ]
quantidade = provaNovo.groupby("idProva").count()
quantidade = quantidade.reset_index()
quantidade = quantidade.drop(['titulo',"data"],axis=1)
quantidade.rename(columns={"idQuestao":"quantidade"}, inplace=True)
provaNovo = pd.merge(left = provaNovo, right = quantidade, on='idProva', how="right")
print(provaNovo.info())
​
Criar o campo ano por prova

[ ]
#Alterando o tipo da variavel ano para datetime
provaNovo["data"] = pd.to_datetime(provaNovo["data"], format = "%Y/%m/%dT%H:%M:%S")
​
provaNovo["ano"], provaNovo["mes"] = provaNovo['data'].dt.year, provaNovo['data'].dt.month
#mergeSubProvas.head()
​
def data(data) :
  mes = data["mes"]
  if mes <= 7 :
    return data["ano"]+0.1
  else:
    return data["ano"]+0.2
provaNovo["ano"] =provaNovo.apply(data, axis=1)
provaNovo = provaNovo.drop("mes",axis = 1)
provaNovo.head()
Criar o campo ano na tabela tabSubProvas

[ ]
tabSubProvas["ano"],tabSubProvas["mes"] = tabSubProvas['data'].dt.year, tabSubProvas['data'].dt.month
tabSubProvas["ano"] = tabSubProvas.apply(data, axis=1)
tabSubProvas = tabSubProvas.drop("mes",axis = 1)
tabSubProvas.head()
Concatenar as tabelas provas e submissao provas

[ ]
mergeSubProvas = pd.merge(right= tabSubProvas, left = provaNovo, on=['idQuestao',"ano"], how="right")
#Só ficar com os registros das provas que o professor criou
mergeSubProvas = mergeSubProvas.loc[mergeSubProvas["idProva_x"] == mergeSubProvas["idProva_y"],:]
mergeSubProvas = mergeSubProvas.drop("idProva_y", axis=1)
mergeSubProvas.rename(columns={"idProva_x":"idProva", "data_x":"dataProva","data_y":"dataSubmissao"}, inplace=True)
mergeSubProvas.info()
[ ]
mergeSubProvas.head()
#Criar o arquivo CSV com as informações das provas e das submissoes

[ ]
mergeSubProvas.to_csv(r'lopSubProva.csv')
Concatenar a tabela mergeUserTurmas e mergeSubProvas

[ ]
mergeSubProvas.rename(columns={"user":"idAluno"}, inplace=True)
mergeUseTurmasSubProva = pd.merge(right= mergeUserTurmas, left = mergeSubProvas, on=['idAluno',"ano"], how="outer")
print (mergeUseTurmasSubProva.shape)
print(mergeUseTurmasSubProva.columns)
mergeUseTurmasSubProva.head(1)
Criar um arquivo CSV com as informações dos usuarios, turmas, submissões e as informações da lista

[ ]
mergeUseTurmasSubProva.to_csv(r'lopUseTurmasSubProva.csv')
Montar um arquivo CSV com as notas de cada aluno por lista

[ ]
#Remover todas as linhas que tem dados faltantes
mergeUseTurmasSubProvaM = mergeUseTurmasSubProva.dropna(axis=0, how='any')
​
Montar um arquivo CSV com as notas de cada aluno por prova
Porcentagem Máxima por aluno em cada questão da prova

[ ]
NotaMaxQuestaoProva = mergeUseTurmasSubProvaM.groupby(["idProva","idAluno","idQuestao"]).max()
NotaMaxQuestaoProva.rename(columns={"porcentagemAcerto":"porcentagemMaxima"}, inplace=True)
NotaMaxQuestaoProva = NotaMaxQuestaoProva.reset_index()
print(NotaMaxQuestaoProva.columns)
NotaMaxQuestaoProva.head(1)
Quantidade de questoes feitas por aluno em cada prova

[ ]
quantidadeProva = NotaMaxQuestaoProva.groupby(["titulo","idAluno"]).count()
quantidadeProva.rename(columns={"quantidade":"questoesFeitas"}, inplace=True)
quantidadeProva = quantidadeProva.reset_index()
quantidadeProva = quantidadeProva[["titulo","idAluno","questoesFeitas"]]
print(quantidadeProva.columns)
quantidadeProva.head(1)
Nota final dos alunos por prova

[ ]
NotaProva = NotaMaxQuestaoProva.groupby(["titulo","idAluno"]).sum()
NotaProva.rename(columns={"porcentagemMaxima":"SomaProva", "quantidade":"questoesFeitas"}, inplace=True)
NotaProva = NotaProva.drop('ano',axis=1)
NotaProva = NotaProva.reset_index()
NotaProva = NotaProva[["titulo",'idAluno','SomaProva']]
print(NotaProva.columns)
NotaProva.head(1)
[ ]
mergeQuanNotaProva =  pd.merge(right= quantidadeProva, left = NotaProva, on=['idAluno','titulo'], how="inner")
print(mergeQuanNotaProva.columns)
print(mergeQuanNotaProva.shape)
mergeQuanNotaProva.head(1)
​
Quantidade de questoes por prova

[ ]
quantQuesProva = provaNovo[['quantidade',"titulo"]]
quantQuesProva = quantQuesProva.drop_duplicates()
print(quantQuesProva.shape)
quantQuesProva.head()
Concatenando as informações das tabelas mergeQuanNotaProva e quantidade para calcular o nota final de cada prova

[ ]
mergeQuanNotaProvaQuanQ = mergeQuanNotaProva.join(quantQuesProva.set_index('titulo'), on='titulo')
mergeQuanNotaProvaQuanQ["notaProva"]= mergeQuanNotaProvaQuanQ["SomaProva"] /mergeQuanNotaProvaQuanQ['quantidade']
print(mergeQuanNotaProvaQuanQ.columns)
print(mergeQuanNotaProvaQuanQ.shape)
mergeQuanNotaProvaQuanQ.head(3)
tabela final dos alunos por nota das provas

[ ]
mergeUseTurmasSubProva.rename(columns={'titulo':"tituloProva","quantidade":"quantidadeProva"}, inplace=True)
mergeUseTurmasSubProvaM = mergeUseTurmasSubProva[["tituloProva",'idAluno', 'ano', 'descricaoTurma', 'idTurma','matricula','nome']]
​
#Renomear as colunas da tabela Listas
mergeQuanNotaProvaQuanQ.rename(columns={'titulo':"tituloProva","questoesFeitas":"questoesFeitasProva","quantidade":"quantidadeProva"}, inplace=True)
mergeQuanNotaProvaQuanM = mergeQuanNotaProvaQuanQ[['tituloProva', 'idAluno','questoesFeitasProva', 'quantidadeProva','notaProva']]
​
tabelaFinalProvas= mergeQuanNotaProvaQuanM.join(mergeUseTurmasSubProvaM.set_index(['tituloProva','idAluno']), on=['tituloProva','idAluno'])
#remover as linhas duplicadas
tabelaFinalProvas = tabelaFinalProvas.drop_duplicates()
tabelaFinalProvas.head()
Exportar o dataFrame para o arquivo CSV de Provas

[ ]
#tabelaFinalT= tabelaFinal.drop('nome',axis=1)
tabelaFinalProvas.to_csv(r'lopUsuarioNotaProva.csv')
Codigo de gerar numero de questoes <100

[ ]
table = pd.read_csv("https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/lopUseTurmasSubLista.csv")
a= table.loc[table["porcentagemAcerto"]<100,"porcentagemAcerto"]
table["nQres"]=a
b = table.groupby("questao")["nQres"].sum().to_frame().reset_index().sort_values(by="nQres")
b.head()
​
Numero de questoes submetidas por aluno

[ ]
table = pd.read_csv("https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/lopUseTurmasSubLista.csv")
nQuestoes = table.groupby(["idAluno","idLista","questao"]).count()
nQuestoes.rename(columns={"quantidade":"nQuestoes"},inplace=True)
nQuestoes=nQuestoes.reset_index()
nQuestoes=nQuestoes[["idAluno","idLista","questao","nQuestoes"]]
nQuestoes.head()
